In [1]:
%matplotlib qt
import scipy.constants as const
from scipy.optimize import curve_fit
from scipy.ndimage import gaussian_filter1d
import h5py
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

from Helpful_functions import *

# cwd = os.getcwd()
cwd = 'C:\\Users\\amorf\\Documents\\Oxford\\DiamondLS\\Wans_Python_Package'

In [2]:
def fermi_func(x, m, c, e, T, bkg_m, bkg_c):
    global resolution
    linear = m*x + c
    a = const.e/const.k
    fermi = 1/(np.exp((x-e)*a/T)+1)
    dx = np.mean(x[1:]-x[:-1])
    sigma = resolution*1e-3/dx
    conv = gaussian_filter1d(fermi*linear, sigma, mode='nearest')
    return conv + bkg_m*x + bkg_c

def Find_Fermi(X, Y):
    global ax
    '''X should be in units of eV'''
    fig, ax = plt.subplots()
    ax.plot(X, Y)
    plt.title('select 2 points to form\linear plot before step')
    plt.pause(0.01)
    P = np.asarray(plt.ginput(2, timeout=-1))
    m = (P[1,1] - P[0,1])/(P[1,0] - P[0,0])
    c = P[1,1] - P[1,0]*m
    plt.title('select a rough guess of\nFermi level')
    plt.pause(0.01)
    P = np.asarray(plt.ginput(1, timeout=-1))
    e = P[0,0]
    plt.title('select 2 points to form\linear plot before step')
    plt.pause(0.01)
    P = np.asarray(plt.ginput(2, timeout=-1))
    bkg_m = (P[1,1] - P[0,1])/(P[1,0] - P[0,0])
    bkg_c = P[1,1] - P[1,0]*bkg_m
    m, c = m - bkg_m, c - bkg_c
    plt.pause(0.01)
    guess = [m,c,e,10,bkg_m,bkg_c]

    ax.plot(X, fermi_func(X, *guess))
    plt.title('This is guess')
    plt.pause(1)

    plt.close()
    fig, ax = plt.subplots()
    ax.plot(X, Y, c='0')
    bounds = ([-np.inf, -np.inf, -np.inf, 0, -np.inf, -np.inf], np.inf)
    fermi_popt, pcov = curve_fit(fermi_func, X, Y, p0 = guess, bounds=bounds, maxfev=10000)
    fermi_perr = np.sqrt(np.diag(pcov))
    ax.plot(X, fermi_func(X,*fermi_popt), color='tab:red')
    ax.axvline(fermi_popt[2], color='0', ls='--', lw=0.8)
    
    m, c, e, T, bkg_m, bkg_c = fermi_popt
    fermi_step = fermi_func(X, m, c, e, T, 0, 0)
    
    F = (max(Y)-min(Y))/max(fermi_step)
    C = min(Y)
    ax.plot(X, fermi_step*F + C, c='tab:blue', ls='--')
    
    plt.title('')
    plt.pause(0.01)
    return fermi_popt, fermi_perr

def Gold_plot(X, Y):
    global ax, fermi_popt, resolution

    fig, ax = plt.subplots()
    plt.title('select point2 which we\'ll fit between')
    ax.plot(X, Y)
    plt.show()
    P = np.asarray(plt.ginput(2, timeout=-1))
    x_mask = (P[0,0]<X)&(X<P[1,0])
    XX, YY = X[x_mask], Y[x_mask]
    plt.close()
    
    resolution = 2.5
    fermi_popt, fermi_perr = Find_Fermi(XX, YY)

    Ef_er = fermi_perr[2]
    m, c, Ef, T, bkg_m, bkg_c = fermi_popt

    '''Coordinates for label'''
    text = f'Fermi E = \n{np.round(Ef, 4)} \u00B1 {round(Ef_er, 4)}\nT = {round(T, 1)}\nsigma={resolution}meV'
    ax.set_yticks([])
    rel_text(ax, 0.75, 0.75, text, fontsize=15)
    plt.pause(0.01)
    
    return fermi_popt

### Load data

In [3]:
file_name = 'i05-157361.nxs'

# use os.chdir to go to where the file is stored
# os.chdir(directory)

with h5py.File(file_name, 'r') as I05:
    T = sigfig(np.squeeze(I05['entry1/sample/cryostat_temperature'][:]), 3)
    data_2D = np.squeeze(I05['entry1/analyser/data'][:]).T
    angles = np.squeeze(I05['entry1/analyser/angles'][:])
    energies = np.squeeze(I05['entry1/analyser/energies'][:])
    
data_2D = dead_pixels(data_2D, 20)

### Find the centre and Femi level

In [4]:
def myZoom(event):
    global fig, ax, xlims, ylims
    if event.key == ' ':
        xlims = ax.get_xlim()
        ylims = ax.get_ylim()
        print(xlims, ylims)
        plt.close()  # Close the current figure
    return xlims, ylims

fig, ax = plt.subplots()
fig.canvas.mpl_connect('key_press_event', myZoom)

ax.contourf(angles, energies, data_2D, 100, cmap='Greys')
plt.title('Zoom to the area for integration\nPress space to finish')
plt.show()

(-0.05698306451612822, 3.709737580645161) (53.494793798701295, 53.565952294372295)


In [5]:
x_mask = (xlims[0]<angles)&(angles<xlims[1])
y_mask = (ylims[0]<energies)&(energies<ylims[1])

X = energies[y_mask]
EDC = np.mean(data_2D[y_mask][:,x_mask], axis=1)

fermi_popt = Gold_plot(X, EDC)

Ef = fermi_popt[2]

energies_eV = (energies - Ef)*1000

In [6]:
# Can edit these lims if you like
xlims = (-7,7)
ylims = (-150, 50)

x_mask = (xlims[0]<angles)&(angles<xlims[1])
y_mask = (ylims[0]<energies_eV)&(energies_eV<ylims[1])

X = angles[x_mask]
Y = energies_eV[y_mask]
Z = data_2D[y_mask][:,x_mask]


def myCentering(event):
    global ax, offset, X, vl, MDC_l2
    offset = event.xdata
    
    vl.set_xdata(offset)
    MDC_l2.set_xdata(2*offset - X)
    plt.pause(0.01)

# Create MDC
MDC_mask = abs(Y)<5
MDC = np.mean(Z[MDC_mask], axis=0)
MDC = MDC/np.max(MDC) * 45

fig, ax = plt.subplots()
fig.canvas.mpl_connect('button_press_event', myCentering)

ax.contourf(X, Y, Z, 100, cmap='Greys')

MDC_l1, = ax.plot(X, MDC, color='tab:blue')
MDC_l2, = ax.plot(-X, MDC, color='tab:red')
vl = ax.axvline(0, color='0', ls='--')

plt.title('Click to move the centering line')
plt.show()

In [7]:
print(Ef)
print(offset)

53.543879763324746
-0.832994129032258
